## Imports

In [11]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import googlemaps
import numpy as np
import re
import os
import folium
from IPython.display import clear_output

## PD Set Options

In [12]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 800)

## Data read-in

In [13]:
df = pd.read_excel('Copy of 230209.2021CCAORetentionPINs.xlsx')

## Data clean

In [17]:
df['PARID'] = df['PARID'].str.replace('-','')

## Test block

In [19]:
sample_pin = '17162050160000'
response = requests.get(f'https://www.cookcountyassessor.com/pin/{sample_pin}')
soup = BeautifulSoup(response.content, 'html.parser')
address = soup.find('span', text='Address').find_next('span').text
print(address)

10 S DEARBORN ST


## Set up scraper

In [23]:
def address_snagger(PARID):
    PIN = PARID['PARID']
    try:
        response = requests.get(f'https://www.cookcountyassessor.com/pin/{PIN}')
        soup = BeautifulSoup(response.content, 'html.parser')
        address = soup.find('span', text='Address').find_next('span').text
        clear_output()
        return address
    except Exception as e:
        clear_output()
        return PIN

In [ ]:
address_list = [address_snagger(row) for row in tqdm(df.to_dict('records'))]

In [84]:
df['ADDRESS'] = address_list

In [85]:
df.to_csv('Assessors_hit_list_with_addresses.csv')

# Make Map

## Geocoder Setup

In [86]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [87]:
### add Cook County, IL to geo_address ###
df['geo_address'] = df['ADDRESS'] + ' Cook County, IL'

In [88]:
### Define and run geocoder ###
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [89]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## HTML Popup Formatter

In [90]:
df.columns

Index(['PARID', '2021 BOR Final \nAssessed Value',
       '2022 CCAO Initial \nAssessed Value', '$ Difference', '% Difference2',
       'Class', 'Township', 'ADDRESS', 'geo_address', 'geocoded', 'lat',
       'lon'],
      dtype='object')

In [91]:
df['2021 BOR Final \nAssessed Value'] = df['2021 BOR Final \nAssessed Value'].apply(lambda x: '{:,.0f}'.format(x))
df['2022 CCAO Initial \nAssessed Value'] = df['2022 CCAO Initial \nAssessed Value'].apply(lambda x: '{:,.0f}'.format(x))
df['$ Difference'] = df['$ Difference'].apply(lambda x: '{:,.0f}'.format(x))
df['% Difference2'] = df['% Difference2'].apply(lambda x: '{:.2f}'.format(x))

In [97]:
def popup_html(row):
    Address = row['ADDRESS']
    BOR_Final_2021 = row['2021 BOR Final \nAssessed Value']
    CCAO_Initial_Assessed_2022 = row['2022 CCAO Initial \nAssessed Value']
    difference = row['$ Difference']
    percent_difference = row['% Difference2']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>Board of Review value: </strong>${}'''.format(BOR_Final_2021) + '''<br>
    <strong>Assessor value: </strong>${}'''.format(CCAO_Initial_Assessed_2022) + '''<br>
    <strong>Difference: </strong>${}'''.format(difference) + '''<br>
    <strong>Percent difference: </strong>{}'''.format(percent_difference) + '''%<br>
    </html>
    '''
    return html

In [110]:
df.to_csv('assessor_hit_list_with_coords_before_corrections.csv')

## Correction Section

In [124]:
### 100 N Riverside St Fix ###
df.at[26,'lat']=(41.884139963130444)
df.at[26,'lon']=(-87.6384974876427)

In [121]:
### Drop rows based on index ###
index_drop_list = [175,214,432,454,463]
df.drop(df.index[index_drop_list], inplace=True)

In [122]:
### Make sure only unknown addresses were dropped ###
df.loc[df['ADDRESS'].str.contains('UNKNOWN')]

,PARID,2021 BOR Final \nAssessed Value,2022 CCAO Initial \nAssessed Value,$ Difference,% Difference2,Class,Township,ADDRESS,geo_address,geocoded,lat,lon


In [126]:
### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=10)


### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"Chicago Tax Reduction Reversals")

m.get_root().html.add_child(folium.Element(title_html))

for index, row in df.iterrows():
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        popup=folium.Popup(popup_html(row), max_width=400))
    marker.add_to(m)

# Display map
m

In [127]:
m.save('index.html')

## Map URL snagger

In [128]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [129]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/assessor_hit_list
